# **HYPOTHESIS TESTING**

## Introduction
This notebook performs hypothesis testing and A/B testing on job market trends using `cleaned_jobs.csv`. 
We aim to test salary variations, skill demand, and experience impact using statistical methods.